# Data exploration

In [3]:
import pandas as pd

df_ground_truth = pd.read_csv('./train.csv')
df_ground_truth.head()

,frame_id,bounding_boxes
0,train/A_001.jpg,0 225 214 317 0 172 345 254 285 240 155 131 70...
1,train/A_002.jpg,0 254 190 293 0 169 338 271 276 238 160 137 70...
2,train/A_003.jpg,0 306 59 241 0 155 306 318 235 233 191 149 713...
3,train/A_004.jpg,0 143 239 298 164 223 240 172 721 293 94 76 57...
4,train/A_005.jpg,0 217 137 270 55 209 323 208 731 296 99 79 573...


In [4]:
print("length of the bounding boxes of image 0: ", len(df_ground_truth["bounding_boxes"][0]))
print("length of the bounding boxes of image 10: ", len(df_ground_truth["bounding_boxes"][10]))

length of the bounding boxes of image 0:  200
length of the bounding boxes of image 10:  214


### Algorithmes

In [5]:
import os
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as patches
from skimage.io import imread


H, W = 720, 1280
N = len(df_ground_truth)


def read_frame(df_annotation, frame):
    """Read frames and create integer frame_id-s"""
    file_path = df_annotation[df_annotation.index == frame]['frame_id'].values[0]
    return imread(file_path)

def annotations_for_frame(df_annotation, frame):
    assert frame in df_annotation.index
    bbs = df_annotation[df_annotation.index == frame].bounding_boxes.values[0]

    if pd.isna(bbs): # some frames contain no vehicles
        return []

    bbs = list(map(lambda x : int(x), bbs.split(' ')))
    return np.array_split(bbs, len(bbs) / 4)

def show_annotation(df_annotation, frame):
    img = read_frame(df_annotation, frame)
    bbs = annotations_for_frame(df_annotation, frame)

    fig, ax = plt.subplots(figsize=(10, 8))

    for x, y, dx, dy in bbs:

        rect = patches.Rectangle((x, y), dx, dy, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

    ax.imshow(img)
    ax.set_title('Annotations for frame {}.'.format(frame))

def bounding_boxes_to_mask(bounding_boxes, H, W):
    """
    Converts set of bounding boxes to a binary mask
    """

    mask = np.zeros((H, W))
    for x, y, dx, dy in bounding_boxes:
        mask[y:y+dy, x:x+dx] = 1

    return mask

def run_length_encoding(mask):

    """
    Produces run length encoding for a given binary mask
    """

    # find mask non-zeros in flattened representation
    non_zeros = np.nonzero(mask.flatten())[0]

    if len(non_zeros) == 0:
        return ''

    padded = np.pad(non_zeros, pad_width=1, mode='edge')

    # find start and end points of non-zeros runs
    limits = (padded[1:] - padded[:-1]) != 1
    starts = non_zeros[limits[:-1]]
    ends = non_zeros[limits[1:]]
    lengths = ends - starts + 1

    return ' '.join(['%d %d' % (s, l) for s, l in zip(starts, lengths)])

In [6]:
bbs = annotations_for_frame(df_ground_truth, 0)
print("#"*20,"image 0","#"*20)
print("BBS", bbs)
print("Raw data:", df_ground_truth["bounding_boxes"][0])

#################### image 0 ####################
BBS [array([  0, 225, 214, 317]), array([  0, 172, 345, 254]), array([285, 240, 155, 131]), array([706, 294,  84,  67]), array([565, 284,  68,  55]), array([1002,  309,   56,   40]), array([678, 293,  48,  44]), array([478, 278,  32,  31]), array([611, 284,  26,  27]), array([436, 280,  41,  26]), array([938, 295, 102,  45]), array([518, 282,  59,  20]), array([1216,  305,   61,   49]), array([428, 284,  18,  15])]
Raw data: 0 225 214 317 0 172 345 254 285 240 155 131 706 294 84 67 565 284 68 55 1002 309 56 40 678 293 48 44 478 278 32 31 611 284 26 27 436 280 41 26 938 295 102 45 518 282 59 20 1216 305 61 49 428 284 18 15


# Scenes 

Each scene has a duration of 202 frames

# Getting empty pictures

In [10]:
path = "train.csv"
df_ground_truth["bounding_boxes"].isna().count()

2222